In [91]:
import pandas as pd
import os
import json

import src.data.movies_char_data as MovieChar
character_df_movies = MovieChar.CharacterData("Character", "character.metadata.tsv", output_name = "character_data_clean.csv")
character_df_movies.clean_raw_data()
character_df_movies.pipeline()
movie_df = MovieChar.MovieData("Movie", "movie.metadata.tsv", output_name = "movie_data_clean.csv")
movie_df.clean_raw_data()
movie_df.pipeline()
from src.utils.names_utils import *


mov_char_data = merge_movies_characters_data(movie_df, character_df_movies)

"""
import src.data.names_data as NamesData
ukNames = NamesData.UKNamesData("UK", "ukbabynames_datasets/ukbabynames.csv")
ukNames.clean_raw_data()
frenchNames = NamesData.FranceNamesData("France", "france.csv", "https://www.insee.fr/fr/statistiques/8205621?sommaire=8205628#dictionnaire", ";")
frenchNames.clean_raw_data()
USNames = NamesData.USNamesData("US", "babyNamesUSYOB-full.csv")
USNames.clean_raw_data()
print(ukNames.head())
print(frenchNames.head())
print(USNames.head())
"""

frenchDf = pd.read_csv("data/clean/names/france.csv")

ukDf = pd.read_csv("data/clean/names/ukbabynames.csv")

usDf = pd.read_csv("data/clean/names/babyNamesUSYOB-full.csv")

## Merge all the names data
names_data = pd.concat([frenchDf, ukDf, usDf], ignore_index=True)

Checking if the data is loaded from a file or from memory
Character : Loading the raw data in the attribute
data/raw/character.metadata.tsv
Character : loaded 450668 rows !
Character : Cleaning the raw data
Character : Loading the raw data in the attribute
data/raw/character.metadata.tsv
Character : loaded 450668 rows !
Character : Cleaning the raw data
Character : Data has been cleaned and saved to data/clean/movies_char/character.metadata.tsv!
Checking if the data is loaded from a file or from memory
Movie : Loading the raw data in the attribute
data/raw/movie.metadata.tsv
Movie : loaded 81740 rows !
Movie : Loading the raw data in the attribute
data/raw/movie.metadata.tsv
Movie : loaded 81740 rows !
Movie : Data has been cleaned and saved to data/clean/movies_char/movie.metadata.tsv!
Wikipedia_movie_ID              int64
Movie_name                     object
Release_date           datetime64[ns]
Revenue                       float64
Runtime               timedelta64[ns]
Languages   

Gathering the N biggest revenue-inducing movies each year

In [92]:
# Creating a new column for the release year (without month and day) of the movie$
df_movies = mov_char_data.clean_df
df_movies['Release_year'] = df_movies['Release_date'].dt.year
# Ditching entries with nan revenue 
df_movies = df_movies.dropna(subset=['Revenue'])

def topNByYear(year,n):
    """
    Returns all characters of the top n movies by revenue in a given year.
    """
    df_year = df_movies[df_movies['Release_year'] == year]
    only_movies = df_year.drop_duplicates(subset=['Wikipedia_movie_ID', 'Revenue'])
    top10 = only_movies.nlargest(n, 'Revenue')

    return df_year[df_year['Wikipedia_movie_ID'].isin(top10['Wikipedia_movie_ID'])]

def isInTopN(name, year,n=10):
    """
    Checks if a name was given to a character in a movie that was in the top n by revenue in a given year.
    If yes, return the corresponding dataset entry, otherwise return None.
    """
    top10 = topNByYear(year,n)
    return top10[top10['Character_name'] == name]

isInTopN("Neo", 1999)



,Wikipedia_movie_ID,Movie_name,Release_date,Revenue,Runtime,Languages,Countries,Genres,Character_name,Actor_DOB,Actor_gender,Actor_height,Actor_ethnicity,Actor_name,Actor_age,Release_year
319733,30007,The Matrix,1999-03-31,463517383.0,0 days 00:00:00.000000135,English,"United States of America, Australia","Thriller, Science Fiction, Adventure, Doomsday...",Neo,1964-09-02,M,1.86,/m/03bkbh,Keanu Reeves,34.0,1999.0


In [108]:
# Taking only the first name of the character name
df_movies['Character_name'] = df_movies['Character_name'].str.split().str[0]

# Taking only characters of the top 100 characters by revenue every year
df_movies = df_movies.groupby('Release_year').apply(lambda x: x.nlargest(100, 'Revenue')).reset_index(drop=True)





df_babies = names_data
merged_df = df_babies.merge(
    df_movies[['Character_name', 'Release_year', 'Movie_name']],
    left_on='Name', right_on='Character_name',
    how='inner'
)

def analyze_trend(group, window_before = 5, window_after = 5):
    movie_name = group['Movie_name'].iloc[0]
    release_year = group['Release_year'].iloc[0]
    pre_movie = group[(group['Year'] < release_year) & (group['Year'] >= release_year - window_before)]
    post_movie = group[(group['Year'] >= release_year) & (group['Year'] <= release_year + window_after)]
    pre_avg = pre_movie['Count'].mean()
    post_avg = post_movie['Count'].mean()
    trend_increase = post_avg - pre_avg
    return pd.Series({'pre_avg': pre_avg, 'post_avg': post_avg, 'trend_increase': trend_increase, 'movie_name': movie_name})

# Apply the function to each character name
trend_df = merged_df.groupby('Character_name').apply(analyze_trend).reset_index()
ranking = trend_df.sort_values('trend_increase', ascending=False)

print(ranking.head(10))


C:\Users\exekr\AppData\Local\Temp\ipykernel_28216\513565364.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_movies = df_movies.groupby('Release_year').apply(lambda x: x.nlargest(100, 'Revenue')).reset_index(drop=True)


    Character_name       pre_avg      post_avg  trend_increase  \
160       Jennifer  11253.800000  29287.083333    18033.283333   
221        Michael  33047.000000  44695.083333    11648.083333   
104          Ethan   4062.200000  10035.000000     5972.800000   
195           Lisa   1419.857143   7222.416667     5802.559524   
327          Tyler   1131.300000   4405.416667     3274.116667   
122          Grace   2218.250000   5263.250000     3045.000000   
320        Tiffany   1107.600000   3067.250000     1959.650000   
247           Paul  10874.800000  12727.416667     1852.616667   
12            Alex   1653.200000   3470.500000     1817.300000   
217           Mary  31979.300000  33744.583333     1765.283333   

                     movie_name  
160                  Love Story  
221                   Peter Pan  
104      Mission: Impossible II  
195                 Rear Window  
327  E.T. the Extra-Terrestrial  
122                  Armageddon  
320        Diamonds Are Forever  
2

C:\Users\exekr\AppData\Local\Temp\ipykernel_28216\513565364.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trend_df = merged_df.groupby('Character_name').apply(analyze_trend).reset_index()
